# Multivariate Normal Distribution

Recall that a random vector $X = (X1,...,X_d)$ has a multivariate normal (or Gaussian) distribution if every linear combination

$$\sum_{i=1}^d{a_iX_i}, \quad a_i \in \mathbb{R}$$

is normally distributed.

The multivariate normal distribution has a joint probability density given by

$$p(x|m,K_0) = (2\pi)^{-d/2}|K_0|^{-1/2}\exp{\left(-\frac{1}{2}(x-m)^TK_0^{-1}(x-m)\right)}$$

where $m \in \mathbb{R}^d$ is the mean vector and $K_0 \in M_d{\mathbb{R}}$ is the (symmetric, positive definite) covariance matrix.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Set parameters

In [3]:
# Define dimension
d = 2

# Set mean vector
m = np.array([1,2]).reshape(2, 1)

# Set covariance function
K_0 = np.array([[2, 1],
              [1, 2]])

Let's us compute the eignvalues of $K_0$

In [4]:
np.linalg.eigvals(K_0)

array([3., 1.])

We see that $K_0$ is indeed positive definite.

# Sampling process

## Step 1: Compute the Cholesky Decomposition

We want to compute the Cholesky decomposition of the covariance matrix $K_0$. That is, we want to find a lower triangular matrix $L \in M_d(\mathbb{R})$ such that $K_0 = LL^T$.

_"In practice it may be necessary to add a small multiple of the identity matrix I to the covariance matrix for numerical reasons. THis is because the eignvalues of the matrix $K_0$ can decay very rapidly and without this stabilization the Cholesky decomposition fails. The effect on the generated samples is to add additional independent noise of variance $\epsilon$. From the context $\epsilon$ can usually be chosen to have inconsequential effects on the samples, while ensuring numerical stability."_

In [5]:
# Define epsilon
epsilon = 0.0001

# Add small perturbation
K = K_0 + epsilon*np.identity(d)

# Cholesky decomposition
L = np.linalg.cholesky(K)
L

array([[1.41424892, 0.        ],
       [0.7070891 , 1.2247959 ]])

Let's verify the desired property:

In [6]:
np.dot(L, np.transpose(L))

array([[2.0001, 1.    ],
       [1.    , 2.0001]])

## Step 2: Generate Independent Samples $u \sim N(0, I)$

In [8]:
# Number of samples
n = 10000

u = np.random.normal(loc=0, scale=1, size=d*n).reshape(d, n)

In [13]:
X = np.random.normal(size=(20, 1))
y = np.random.normal(size=(1,20))

In [14]:
np.dot(X, y).shape

(20, 20)

In [15]:
X

array([[ 1.81930882e+00],
       [ 1.01007447e+00],
       [ 6.50153054e-01],
       [ 1.44090866e+00],
       [-7.00952244e-04],
       [ 2.17090062e-01],
       [ 1.79852239e-01],
       [-1.18573790e+00],
       [ 7.36682392e-01],
       [ 1.21790330e+00],
       [-1.35633942e+00],
       [ 7.86352698e-01],
       [ 1.29793477e+00],
       [-1.46870230e+00],
       [-2.25664491e+00],
       [-2.59308535e-01],
       [ 2.73818729e-02],
       [ 1.71951114e+00],
       [ 1.04882965e+00],
       [-1.27520753e-03]])

In [16]:
y

array([[-0.13973666, -0.25653674, -2.31180254,  0.66512936, -2.41577328,
        -1.05016128, -0.2060651 , -1.89463288, -0.99203151,  0.83642052,
        -0.09990854, -0.10374076,  2.07595023,  1.60844712, -0.99828925,
         1.51044254, -0.77379085, -0.45627961, -1.16042524, -0.51613431]])